In [1]:
from bia_bob._rag import scan_plugins
import pandas as p#
import yaml

In [2]:
import importlib_metadata
all_modules = importlib_metadata.packages_distributions()

In [3]:
from importlib.metadata import entry_points
try:
    bia_bob_plugins = entry_points(group='bia_bob_plugins')
except TypeError:
    all_plugins = entry_points()
    try:
        bia_bob_plugins = all_plugins['bia_bob_plugins']
    except KeyError:
        bia_bob_plugins = []

def get_package_version(package_name):
    from importlib.metadata import version
    return version(package_name)

def embed(text):
    from openai import OpenAI
    client = OpenAI()
    print("embedding", text)

    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

db_dict = {
    "package":[],
    "hint":[],
    "vector":[],
}
df_dict = pd.DataFrame(db_dict)

store_filename = "bia_bob_vectore_store.yaml"

import os
if os.path.exists(store_filename):
    #df_dict = pd.read_csv(store_filename)
    with open(store_filename, mode="rt", encoding="utf-8") as test_df_to_yaml:
        df_dict = pd.DataFrame(yaml.full_load(test_df_to_yaml)['vectorstore'])
    db_dict = df_dict.to_dict(orient="list")

In [4]:
df_dict

,hint,package,vector
0,* Displays an image with a slider and label sh...,stackview==0.7.6,"[-0.014895317144691944, 0.0096043162047863, -0..."
1,* Allows cropping an image along all axes....,stackview==0.7.6,"[-0.0022624426055699587, 0.004129345063120127,..."
2,* Showing an image stored in variable `ima...,stackview==0.7.6,"[-0.015545042231678963, -0.019260738044977188,..."
3,* Display an image stored in a variable `i...,stackview==0.7.6,"[-0.01666620932519436, -0.03690773993730545, -..."
4,* Build a user interface with sliders for ...,stackview==0.7.6,"[-0.036892376840114594, 0.009743290953338146, ..."
5,* Display an image and allows slicing in t...,stackview==0.7.6,"[-0.0005710396799258888, -0.017563432455062866..."
6,* Display an image with a slider and label...,stackview==0.7.6,"[0.004523767624050379, -0.015061167068779469, ..."
7,"* Display two images side by side, with an...",stackview==0.7.6,"[-0.0035302755422890186, -0.04014468193054199,..."
8,* Display an image with a slider to naviga...,stackview==0.7.6,"[-0.0005727233947254717, -0.029088573530316353..."
9,* Allows switching between multiple images...,stackview==0.7.6,"[0.01182166300714016, -0.02974669262766838, -0..."


In [5]:
for b in bia_bob_plugins:
    module_name = b.value.split(".")[0]
    package_name = all_modules[module_name][0]
    package_version = get_package_version(package_name)
    
    package_name_and_version = package_name + "==" + package_version
    
    # check if these functions are in the hint-vector store already. If not: add them
    if df_dict[df_dict["package"] == package_name_and_version].size == 0: 
        print("BiA-Bob scanning", package_name_and_version)
        func = b.load()

        # load instructions from a plugin
        instructions = func().replace("\n    \n", "\n\n").strip().split("\n\n")

        for i in instructions:
            e = embed(i)
            
            db_dict["package"].append(package_name_and_version)
            db_dict["hint"].append(i)
            db_dict["vector"].append(e)

            


In [6]:
df_dict = pd.DataFrame(db_dict)
with open(store_filename, 'w') as file:
    documents = yaml.dump({'vectorstore': df_dict.to_dict(orient='records')}, file, default_flow_style=False)

In [7]:
df_dict[['vector', 'hint']]

,vector,hint
0,"[-0.014895317144691944, 0.0096043162047863, -0...",* Displays an image with a slider and label sh...
1,"[-0.0022624426055699587, 0.004129345063120127,...",* Allows cropping an image along all axes....
2,"[-0.015545042231678963, -0.019260738044977188,...",* Showing an image stored in variable `ima...
3,"[-0.01666620932519436, -0.03690773993730545, -...",* Display an image stored in a variable `i...
4,"[-0.036892376840114594, 0.009743290953338146, ...",* Build a user interface with sliders for ...
5,"[-0.0005710396799258888, -0.017563432455062866...",* Display an image and allows slicing in t...
6,"[0.004523767624050379, -0.015061167068779469, ...",* Display an image with a slider and label...
7,"[-0.0035302755422890186, -0.04014468193054199,...","* Display two images side by side, with an..."
8,"[-0.0005727233947254717, -0.029088573530316353...",* Display an image with a slider to naviga...
9,"[0.01182166300714016, -0.02974669262766838, -0...",* Allows switching between multiple images...
